# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics1 import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('C:\\Users\\mmingalov\\geekbrains-recommend-systems\\retail_train.csv')
item_features = pd.read_csv('C:\\Users\\mmingalov\\geekbrains-recommend-systems\\product.csv')
user_features = pd.read_csv('C:\\Users\\mmingalov\\geekbrains-recommend-systems\\hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [4]:
recommender = MainRecommender(data_train_lvl_1)

In [5]:
recommender.get_als_recommendations(2375, N=200)

[899624,
 1044078,
 871756,
 844179,
 1106523,
 1046545,
 5569230,
 865456,
 944534,
 12301839,
 832678,
 854852,
 1051323,
 1116376,
 1081177,
 883932,
 1004906,
 870547,
 965766,
 5585510,
 1068719,
 8090537,
 1004390,
 925862,
 963727,
 5568378,
 999714,
 823704,
 1123146,
 1000753,
 823990,
 835530,
 9836106,
 9835223,
 952163,
 1097458,
 1022428,
 965267,
 888543,
 937292,
 999779,
 1096635,
 850102,
 1029743,
 8065410,
 999858,
 863447,
 1000736,
 1134678,
 8090521,
 896613,
 828106,
 839818,
 1096261,
 999270,
 9836195,
 847790,
 995896,
 850925,
 1059902,
 1100972,
 832442,
 861279,
 1091383,
 1099905,
 6534480,
 990797,
 5568197,
 896862,
 12731685,
 930666,
 841220,
 826666,
 947858,
 829138,
 12352293,
 1009770,
 12262978,
 825343,
 12302069,
 9526410,
 1138467,
 1079067,
 1037863,
 1038663,
 1070702,
 1094833,
 6463729,
 827919,
 865528,
 1036347,
 1025535,
 1042907,
 821741,
 1003616,
 1021324,
 916122,
 880150,
 1002850,
 948650,
 1012587,
 12731432,
 972143,
 1107420,
 5

In [6]:
recommender.get_own_recommendations(2375, N=200)

[948640,
 918046,
 847962,
 907099,
 873980,
 884694,
 10285454,
 1107760,
 7169090,
 979674,
 10308345,
 1069531,
 974766,
 1015474,
 950935,
 847066,
 1102207,
 1020770,
 9521787,
 974265,
 940996,
 8019845,
 5567194,
 12811490,
 1003616,
 973181,
 890719,
 982955,
 9677152,
 998519,
 1072685,
 1131382,
 1021715,
 12263119,
 960791,
 7441873,
 986021,
 956666,
 1038692,
 9677748,
 9297223,
 927030,
 12757653,
 1046919,
 6391532,
 989069,
 1068451,
 951954,
 835300,
 937343,
 1047249,
 13876348,
 1061732,
 981601,
 1121028,
 1087547,
 828393,
 996269,
 951951,
 1036093,
 1023815,
 5570408,
 827667,
 1082454,
 1006878,
 5570048,
 841309,
 1078652,
 1115553,
 1056492,
 1138467,
 1004945,
 947858,
 1092885,
 1121694,
 938138,
 8019916,
 827919,
 984315,
 10341855,
 883932,
 8291322,
 1096794,
 1028938,
 1087618,
 8020166,
 1082185,
 866871,
 930666,
 825994,
 910151,
 823990,
 848029,
 896613,
 12301839,
 1117219,
 1135258,
 869868,
 1046545,
 899624,
 6442594,
 1137775,
 825343,
 104290

In [7]:
recommender.get_similar_items_recommendation(2375, N=200)

[1046545,
 1044078,
 1042907,
 5568732,
 1133312,
 999999,
 999104,
 896613,
 885863,
 1025535,
 1044078,
 878996,
 906923,
 871611,
 996087,
 919681,
 828106,
 933835,
 1021522,
 990335,
 1070702,
 999104,
 9835606,
 12301100,
 899624,
 6423993,
 1014116,
 5564303,
 985999,
 1026946,
 9835903,
 1127025,
 1000753,
 992826,
 8011291,
 920200,
 918994,
 825343,
 1022428,
 912704,
 1055425,
 916122,
 869322,
 929768,
 894627,
 12648296,
 1110244,
 904105,
 937292,
 13877192,
 954651,
 1104349,
 5570513,
 898068,
 904574,
 1002771,
 844179,
 828106,
 883068,
 1105488,
 969977,
 825343,
 1120559,
 1026118,
 899459,
 9836195,
 8090537,
 882247,
 5568378,
 948650,
 918994,
 12262778,
 1098435,
 10309725,
 1079338,
 969945,
 870428,
 890909,
 1133312,
 858700,
 937292,
 950575,
 999858,
 5565202,
 9297055,
 913278,
 1057231,
 15778319,
 949345,
 865456,
 908236,
 12757425,
 958382,
 998119,
 1048507,
 856827,
 1095246,
 1123842,
 823704,
 985181,
 994995,
 1087102,
 7024871,
 1083856,
 1103105

In [8]:
recommender.get_similar_users_recommendation(2375, N=200)

[974265,
 10457044,
 837969,
 12523928,
 896757,
 9245108,
 1038745,
 12427353,
 1107760,
 994577,
 820612,
 963686,
 921406,
 945909,
 7410021,
 875392,
 1118946,
 1101502,
 9553048,
 5572803,
 892728,
 865757,
 873324,
 873044,
 992730,
 1037135,
 1081533,
 995502,
 1118120,
 959830,
 894360,
 983665,
 1057168,
 822970,
 9837092,
 937736,
 839605,
 917033,
 12731544,
 918638,
 5571261,
 874563,
 853124,
 1120190,
 8066803,
 1028422,
 1133850,
 948225,
 873715,
 1096573,
 951834,
 5569309,
 1102003,
 830686,
 1056212,
 948239,
 9392700,
 997025,
 950894,
 1093490,
 825317,
 9553335,
 1116050,
 1124971,
 10198378,
 979674,
 938118,
 12262832,
 1105426,
 1068865,
 1128477,
 1012801,
 1030093,
 1117219,
 841365,
 1028238,
 12523928,
 947412,
 979674,
 852015,
 965956,
 1138596,
 837495,
 927028,
 820923,
 902640,
 1131625,
 1012801,
 969866,
 1129805,
 1097398,
 5707857,
 8090542,
 1042571,
 959455,
 1055403,
 823031,
 12524016,
 1071196,
 1089986,
 5569135,
 1023160,
 920025,
 6544382,


### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [10]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [11]:
result_lvl_1['ALS_recs'] = 0
result_lvl_1['ALS_recs'] = result_lvl_1['ALS_recs'].astype(object)
# пока оставляю через исключение, как сделала изначально, чтобы видеть все в одном файле (с налету не получилось переделать)
for i in range(result_lvl_1.shape[0]):
    try:
        result_lvl_1.at[i, 'ALS_recs'] = recommender.get_als_recommendations(result_lvl_1['user_id'][i], N=200)
    except IndexError:
        # если пользователя нет в списке, рекомендую популярные товары:
        result_lvl_1.at[i, 'ALS_recs'] = []
        result_lvl_1.at[i, 'ALS_recs'] = recommender._extend_with_top_popular(result_lvl_1['ALS_recs'][i], N=200)
result_lvl_1.head(2)

,user_id,actual,ALS_recs
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[885290, 1094924, 1028238, 824758, 1047619, 95..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[5569230, 1104414, 1127831, 1029743, 866211, 8..."


In [12]:
ALS_recall = result_lvl_1.apply(lambda row: recall_at_k(row['ALS_recs'], row['actual'], k=200), axis=1).mean()
ALS_recall

0.09689530320579752

In [13]:
result_lvl_1['own_recs'] = 0
result_lvl_1['own_recs'] = result_lvl_1['own_recs'].astype(object)
for i in range(result_lvl_1.shape[0]):
    try:
        result_lvl_1.at[i, 'own_recs'] = recommender.get_own_recommendations(result_lvl_1['user_id'][i], N=200)
    except ValueError:
        result_lvl_1.at[i, 'own_recs'] = []
        result_lvl_1.at[i, 'own_recs'] = recommender._extend_with_top_popular(result_lvl_1['own_recs'][i], N=200)
result_lvl_1.head(2)

,user_id,actual,ALS_recs,own_recs
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[885290, 1094924, 1028238, 824758, 1047619, 95...","[856942, 9297615, 5577022, 877391, 9655212, 88..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[5569230, 1104414, 1127831, 1029743, 866211, 8...","[911974, 1076580, 1103898, 5567582, 1056620, 9..."


In [14]:
own_recall = result_lvl_1.apply(lambda row: recall_at_k(row['own_recs'], row['actual'], k=200), axis=1).mean()
own_recall

0.1352815146098268

In [15]:
%%time
result_lvl_1['similar_items_recs'] = 0
result_lvl_1['similar_items_recs'] = result_lvl_1['similar_items_recs'].astype(object)
for i in range(result_lvl_1.shape[0]):
    result_lvl_1.at[i, 'similar_items_recs'] = recommender.get_similar_items_recommendation(result_lvl_1['user_id'][i], N=200)
    
result_lvl_1.head(2)

Wall time: 36.3 s


,user_id,actual,ALS_recs,own_recs,similar_items_recs
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[885290, 1094924, 1028238, 824758, 1047619, 95...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[824758, 826597, 5570513, 5577022, 5566697, 98..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[5569230, 1104414, 1127831, 1029743, 866211, 8...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[8090537, 5569845, 1044078, 985999, 880888, 81..."


In [16]:
similar_items_recall = result_lvl_1.apply(lambda row: recall_at_k(row['similar_items_recs'], row['actual'], k=200), axis=1).mean()
similar_items_recall

0.08626387532033405

In [ ]:
%%time
# занимает около получаса!!!
result_lvl_1['similar_users_recs'] = 0
result_lvl_1['similar_users_recs'] = result_lvl_1['similar_users_recs'].astype(object)
for i in range(result_lvl_1.shape[0]):
    try:
        result_lvl_1.at[i, 'similar_users_recs'] = recommender.get_similar_users_recommendation(result_lvl_1['user_id'][i], N=200)
    except IndexError:
        result_lvl_1.at[i, 'similar_users_recs'] = []
        result_lvl_1.at[i, 'similar_users_recs'] = recommender._extend_with_top_popular(result_lvl_1['similar_users_recs'][i], N=200)
    except ValueError:
        result_lvl_1.at[i, 'similar_users_recs'] = []
        result_lvl_1.at[i, 'similar_users_recs'] = recommender._extend_with_top_popular(result_lvl_1['similar_users_recs'][i], N=200)
result_lvl_1.head(2)

In [ ]:
similar_users_recall = result_lvl_1.apply(lambda row: recall_at_k(row['similar_users_recs'], row['actual'], k=200), axis=1).mean()
similar_users_recall

Видим, что наилучший recall у предсказания по собственным покупкам, затем ALS, потом рекомендации по cхожести товаров. Рекомендации по схожим пользователям считаются очень долго и дают минимальный результат.

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
# your_code

### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - money precision@5. Порог для уcпешной сдачи проекта money precision@5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- **2 новых товара** (юзер никогда не покупал)
- **1 дорогой товар, > 7 долларов**
- **Все товары из разных категорий** (категория - department)  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)